In [35]:
%matplotlib inline
from pylab import *
import pandas as pd
from sklearn.decomposition import PCA
import yaml

In [36]:
def sort_lidar_file_and_shape(lidar_file_name, yaml_file_name, width=2088, height=64):
    df = pd.read_csv(lidar_file_name,
                     names = ['x', 'y', 'z', 'intensity', 'ring', 'rotation', 'revolution'])
    
    yaml_file = yaml.load(open(yaml_file_name))
    
    calibration = yaml_file['lasers']
    
    sorted_lasers = sorted(calibration, key=lambda x: x['vert_correction'], reverse=True)

    for i in range(0, 64):
        df.loc[df['ring'] == i, 'rotation'] = (df.loc[df['ring'] == i, 'rotation'] 
                                               + sorted_lasers[i]['rot_correction'] * 18000 / np.pi)
    
    df.loc[df['rotation'] > 36000, 'rotation'] -= 36000
    df.loc[df['rotation'] < 0, 'rotation'] += 36000
    
    img = np.zeros((height, width, 3))
    for i in range(height):
        img[i] = df.loc[df['ring'] == i].sort_values(['rotation']).as_matrix()[:, :3]
    
    return img

In [37]:
readings_file_name = "../1504892561136210918-cloudpoint.csv"
calibration_file_name = "64HDL_S2.yaml"

lidar_table = sort_lidar_file_and_shape(readings_file_name, calibration_file_name)

width = 2088
height = 64 
lidar = lidar_table.reshape((width * height, 3))

In [38]:
#dists = np.linalg.norm(lidar, axis=-1)
#inds = np.where(dists < 10)

inds = np.arange(len(lidar))
 
lidar_center = lidar[inds]

green = np.array([0, 1, 0])
red = np.array([1, 0, 0])
blue = np.array([0, 0, 1])

In [39]:
lidar_file_name = "lidar_test.csv"

with open(lidar_file_name, 'w') as f:
    for rd in lidar_center:
        f.write("{}, {}, {}\n".format(rd[0], rd[1], rd[2]))

In [40]:
order = np.zeros((height, width, 3))
half_width = width / 2
for i in range(width):
    for j in range(height):
        x = i
#         if x >= half_width:
#             x = width - i
#         rgval = x / half_width
        rgval = x / width
        bval = j / height
        order[j, i] = rgval * red + (1 - rgval) * green + bval * blue

order = order.reshape((width * height, 3))

order_center = order[inds]

with open("width_order_colors_test.csv", 'w') as f:
    for rd in order_center:
        f.write("{}, {}, {}\n".format(rd[0], rd[1], rd[2]))

In [41]:
depths = np.linalg.norm(lidar_table, axis=-1)

In [66]:
normals = np.zeros(lidar_table.shape)
neighbor_y_radius = 3

neighbor_x_radius = 3

#
for i in range(neighbor_y_radius, height-neighbor_y_radius - 1):
    for j in range(0, width):        
        neighbor_y_start = i - neighbor_y_radius
        neighbor_y_end = i + neighbor_y_radius + 1
        neighbor_x = np.arange(j - neighbor_x_radius, j + neighbor_x_radius + 1)
        
        neighbors = lidar_table[neighbor_y_start:neighbor_y_end]
        neighbors = np.take(neighbors, neighbor_x, axis = 1, mode='wrap')
        neighbors = neighbors.reshape(((neighbor_y_radius * 2 + 1) * (neighbor_x_radius * 2 + 1), 3))
        
        neighbor_depths = depths[neighbor_y_start:neighbor_y_end]
        neighbor_depths = np.take(neighbor_depths, neighbor_x, axis=1, mode='wrap')
        neighbor_depths = neighbor_depths.ravel()
        
        valid_neighbors = neighbors[(neighbor_depths - depths[i, j]) <= 0.2]
        if valid_neighbors.shape[0] >= 3:
            covariance = np.cov(valid_neighbors, rowvar=False)
            eigvals, eigvecs = np.linalg.eigh(covariance)
            normals[i, j] = eigvecs[:, 0]
            normals[i, j] /= np.linalg.norm(normals[i, j])

normal_img = normals / 2.0 + 0.5
normals = normals.reshape((normals.shape[0] * normals.shape[1], 3))

In [67]:
normal_file_name = "normal_colors_test.csv"

normals_center = normals[inds]
with open(normal_file_name, 'w') as f:
    temp = normals_center / 2.0 + 0.5
    for rd in temp:
        f.write("{}, {}, {}\n".format(rd[0], rd[1], rd[2]))

In [68]:
normal_ply_file_name = "normals.ply"

with open(normal_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(len(normals_center)))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("property float32 nx\n")
    f.write("property float32 ny\n")
    f.write("property float32 nz\n")
    f.write("end_header\n")
    for i in range(len(lidar_center)):
        f.write("{} {} {} {} {} {}\n".format(lidar_center[i][0], lidar_center[i][1], lidar_center[i][2],
                                             normals_center[i][0], normals_center[i][1], normals_center[i][2]))

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4))
ax.imshow(np.rot90(np.rot90(normal_img)))